In [1]:
# using Distributed; addprocs(Sys.CPU_THREADS);
using Statistics
using DataStructures
using DataFrames
using NNlib
using Graphs
using PyCall
nx = pyimport("networkx")
pyg = pyimport("torch_geometric")
include("src/weisfeiler_lehman.jl")
include("src/mpgnn.jl")
include("src/utils.jl")
colors = theme_palette(:default)

## WL and MPGNN basic test

In [9]:
edge_index = [
     1 1 1 2 2 3 3 3 4 4 5 5;
     2 3 5 1 3 1 2 4 3 5 1 4
]
x₀ = [
     1, 1, 1, 1, 1
]
γ = 0.1
wl_iters, wl_coloring = WeisfeilerLehman(x₀, edge_index)
gnn_coloring = MPGNN(x₀, edge_index, γ, 3)

println(wl_coloring)
println(gnn_coloring)

BigFloat[1.0, 2.0, 1.0, 3.0, 3.0]
BigFloat[1.0, 2.0, 1.0, 3.0, 3.0]


![Alt Text](wl.gif)

## 1. Bag of Graphs Experiment
### 1.1 Bag of Erdős–Rényi

Here, we create an Erdős–Rényi random graph with **n** vertices. Edges are added between pairs of vertices with probability **p**.

In [31]:
setprecision(BigFloat, 50)
γs = sort([rand(BigFloat) for _ in 1:50])
lottery_γs_erdos = []
success_sum = []
P = [p for p in 0.05 : 0.01 : 0.1]
N = [n for n in 20 : 5 : 265]

graphs = [[erdos_renyi(N[j], P[i]) for j in 1:length(N)] for i ∈ 1:length(P)]
graphs = [[graphs[i][j][findall(>(0), degree(graphs[i][j]))] for j in 1:length(N)] for i ∈ 1:length(P)]

for γ ∈ γs
    success_matrix = zeros(length(P), length(N))
    for i ∈ 1:length(P)
        for j in 1:length(N)
            success_matrix[i, j] = WL_vs_GNN(graphs[i][j], γ)
        end
    end
    push!(success_sum, sum(success_matrix))
    if sum(success_matrix) == sum(ones(length(P), length(N)))
        # println("lottery ticket γ: $(γ)")
        push!(lottery_γs_erdos, γ) 
    end
end

In [45]:
lottery_γs_erdos

44-element Vector{Any}:
 0.021563476993270214
 0.052744037230915808
 0.057323206991157782
 0.063377716953294083
 0.07405842412748509
 0.085557881522405843
 0.14612235643383098
 0.14953867364812989
 0.1528851789703225
 0.16130093767367271
 ⋮
 0.68160009812719213
 0.68250860024421645
 0.72272678918125166
 0.77846219579510745
 0.78629000028250218
 0.81200835123464987
 0.82204473295555669
 0.82676212175756092
 0.89197869656456952

In [ ]:
ones_sum = sum(ones(length(P), length(N)))
selected_lottery_γs = vcat(lottery_γs_erdos[1], vcat(rand(lottery_γs_erdos, 3), lottery_γs_erdos[end]))
plot(γs, [ones_sum for _ ∈ γs], label="WL", dpi=1000)
plot!(γs, success_sum, label="GNN")
scatter!(lottery_γs_erdos, [ones_sum for _ ∈ lottery_γs_erdos],
        mc=colors[3], ms=3, ma=0.5,
        label="Lottery γs")
scatter!(selected_lottery_γs, [ones_sum for _ ∈ selected_lottery_γs], 
        series_annotations = text.(selected_lottery_γs, :left, 8, rotation=270), 
        mc=colors[3], ms=3, ma=0.5,
        label="")

ylims!(0, 1.1*ones_sum)
title!("WL vs 1D-MPNN, Erdős Rényi")
xlabel!("γ")
ylabel!("Successful Simulations")
savefig("results\\1.1.erdos_renyi.png")

### 1.2 Bag of Barabási–Albert

Here, we create a Barabási–Albert model random graph with n vertices. It is grown by adding new vertices to an initial graph with k vertices. Each new vertex is attached with k edges to k different vertices already present in the system by preferential attachment. Initial graphs are undirected and consist of isolated vertices by default.

In [28]:
setprecision(BigFloat, 135)
γs = sort([rand(BigFloat) for _ in 1:50])
lottery_γs = []
success_sum = []
K = [k for k in 5 : 5 : 20]
N = [n for n in 50 : 3 : 273]

graphs = [[barabasi_albert(N[j], K[i]) for j in 1:length(N)] for i ∈ 1:length(K)]
graphs = [[graphs[i][j][findall(>(0), degree(graphs[i][j]))] for j in 1:length(N)] for i ∈ 1:length(K)]

for γ ∈ γs
    success_matrix = zeros(length(K), length(N))
    for i ∈ 1:length(K)
        for j in 1:length(N)
            success_matrix[i, j] = WL_vs_GNN(graphs[i][j], γ)
        end
    end
    push!(success_sum, sum(success_matrix))
    if sum(success_matrix) == sum(ones(length(K), length(N)))
        # println("lottery ticket γ: $(γ)")
        push!(lottery_γs, γ) 
    end
end

In [44]:
lottery_γs

44-element Vector{Any}:
 0.00911638695000082373421465944185524007114575
 0.0137436070323187821989066193812732161940691
 0.0620895576857160953960370035055580717843639
 0.0828300999503917918730639016777337346565403
 0.083923602889189387085363244392360297735704
 0.0867372744267148347172077895010847675508327
 0.0959802447810247050342796978840170365775362
 0.102576920316872050603558149390988771995365
 0.141440907585404652455139703282799384808581
 0.16638417074215462209194198545596909631458
 ⋮
 0.762950370321839315125960393090576428400023
 0.785797965724755647440686230144460925333673
 0.788977586462408109470817902828955796838461
 0.800424027438892389136964867592630284205623
 0.822733749241030932771698052632946783748034
 0.850131818606531642027651181579492146381754
 0.872159371117777358408965359667590920586228
 0.875317148656477275606322478385453622669189
 0.87841386771056150819276869601179928650411

In [ ]:
setprecision(BigFloat, 135)
ones_sum = sum(ones(length(K), length(N)))
selected_lottery_γs = vcat(lottery_γs[1], vcat(rand(lottery_γs, 3), lottery_γs[end]))
plot(γs, [ones_sum for _ ∈ γs], label="WL", dpi=1000)
plot!(γs, success_sum, label="GNN")
scatter!(lottery_γs, [ones_sum for _ ∈ lottery_γs],
        mc=colors[3], ms=3, ma=0.5,
        label="Lottery γs")
scatter!(selected_lottery_γs, [ones_sum for _ ∈ selected_lottery_γs], 
        series_annotations = text.(BigFloat.(selected_lottery_γs), :left, 8, rotation=270), 
        mc=colors[3], ms=3, ma=0.5,
        label="")

ylims!(0, 1.1*ones_sum)
title!("WL vs 1D-MPNN, Barabási Albert")
xlabel!("γ")
ylabel!("Successful Simulations")
savefig("results\\1.2.barabasi_albert.png")

## 2. Successful Bits Experiment 

In [48]:
# P = [p for p in 0.1 : 0.05 : 0.5]
P = [0.1]
N = [n for n in 10 : 50 : 3010]

graphs = [[erdos_renyi(N[j], P[i]) for j in 1:length(N)] for i ∈ 1:length(P)]
graphs = [[graphs[i][j][findall(>(0), degree(graphs[i][j]))] for j in 1:length(N)] for i ∈ 1:length(P)]

bits = [i for i ∈ 2: 1 :400]
bits_success = [[] for _ in N]

setprecision(BigFloat, 100)
# γs = [rand(BigFloat)*0.8 for _ in 1:20]

for i ∈ 1:length(P)
    for j in 1:length(N)
        x₀, edge_index = construct_graph(graphs[i][j])
        iters, coloring_wl = WeisfeilerLehman(x₀, edge_index)
        for sample ∈ 1:20
            for bit ∈ bits
                setprecision(BigFloat, bit)
                γ = rand(BigFloat)
                coloring_gnn = MPGNN(x₀, edge_index, γ, iters)
                if sum(coloring_wl) <= sum(coloring_gnn)
                    push!(bits_success[j], bit)
                    break
                end
            end
        end
    end
end

In [ ]:
regression = Regression([log2(n) for n in N], mean.(bits_success))
plot(N, mean.(bits_success), grid=false, yerror=std.(bits_success), label="Avg", dpi=1000)
plot!(N, [regression[1] + regression[2]*log2(n) for n in N], linestyle=:dash, label="$(round(regression[2], digits=1))log(n)")
# for i in 1:length(bits_success)
#     scatter!([N[i] for _ in 1:length(bits_success[i])], bits_success[i], label="")
# end

title!("Model Size vs Input Size, Erdős Rényi")
xlabel!("Graph Size")
ylabel!("Precision Bits")
savefig("results\\2.bits_erdos_renyi.png")

## 3. Big Erdős Rényi Experiment

In [11]:
graph = erdos_renyi(5000, 0.002)
graph = graph[findall(>(0), degree(graph))]
x₀, edge_index = construct_graph(graph)

iters, coloring_wl = WeisfeilerLehman(x₀, edge_index)
wl_classes = maximum(coloring_wl)
println("nodes: $(nv(graph)), WL classes: $(wl_classes)")

nodes: 5000, WL classes: 3718.0


In [12]:
setprecision(BigFloat, 100)
bits = [i for i ∈ 10: 1: 40]
γs = sort([rand(BigFloat) for _ in 1:20])
classes = [[] for bit in bits]

for (i, bit) ∈ enumerate(bits)
    setprecision(BigFloat, bit)
    # γs = sort([rand(BigFloat) for _ in 1:20])
    for γ ∈ γs
        coloring_gnn = MPGNN(x₀, edge_index, γ, iters)
        push!(classes[i], maximum(coloring_gnn))
    end
end

In [ ]:
plot(bits, [wl_classes for _ in bits], label="WL", dpi=1000)
plot!(bits, mean.(classes), grid=false, label="GNN")
for i in 1:2:length(classes)
    scatter!([bits[i] for _ in 1:length(classes[i])], classes[i], label="")
end
title!("Model Size vs Performance, Big Erdős Rényi")
xlabel!("Precision Bits")
ylabel!("Number of classes")
savefig("results\\3.big_erdos_renyi.png")

## 4. Cora Experiment

In [14]:
dataset_directory = "data/"
dataset = pyg.datasets.Planetoid(dataset_directory, "Cora")
graph = dataset[0]
x₀, edge_index = ones(2708), graph.edge_index.numpy() .+ 1

iters, coloring_wl = WeisfeilerLehman(x₀, edge_index)
wl_classes = maximum(coloring_wl)
println("nodes: $(2708), WL classes: $(wl_classes)")

nodes: 2708, WL classes: 2365.0


In [15]:
setprecision(BigFloat, 60)
bits = [i for i ∈ 10: 1 :40]
γs = sort([rand(BigFloat) for _ in 1:20])
classes = [[] for bit in bits]

for (i, bit) ∈ enumerate(bits)
    setprecision(BigFloat, bit)
    # γs = sort([rand(BigFloat) for _ in 1:10])
    for γ ∈ γs
        coloring_gnn = MPGNN(x₀, edge_index, γ, iters)
        push!(classes[i], maximum(coloring_gnn))
    end
end

In [ ]:
plot(bits, [wl_classes for _ in bits], label="WL", dpi=1000)
plot!(bits, mean.(classes), grid=false, label="GNN")
for i in 1:2:length(classes)
    scatter!([bits[i] for _ in 1:length(classes[i])], classes[i], label="")
end
title!("Model Size vs Performance, Cora")
xlabel!("Precision Bits")
ylabel!("Number of classes")
savefig("results\\4.cora.png")

## 5. WL-completeness ratio

In [2]:
dataset_directory = "data/"

AIDS_dataset = pyg.datasets.TUDataset(dataset_directory, "AIDS")
BZR_dataset = pyg.datasets.TUDataset(dataset_directory, "BZR")
COX2_dataset = pyg.datasets.TUDataset(dataset_directory, "COX2")
DHFR_dataset = pyg.datasets.TUDataset(dataset_directory, "DHFR")
DD_dataset = pyg.datasets.TUDataset(dataset_directory, "DD")
ENZYMES_dataset = pyg.datasets.TUDataset(dataset_directory, "ENZYMES")
FRANKENSTEIN_dataset = pyg.datasets.TUDataset(dataset_directory, "FRANKENSTEIN")
IMDB_BINARY_dataset = pyg.datasets.TUDataset(dataset_directory, "IMDB-BINARY")
IMDB_MULTI_dataset = pyg.datasets.TUDataset(dataset_directory, "IMDB-MULTI")
MSRC_9_dataset = pyg.datasets.TUDataset(dataset_directory, "MSRC_9")
MSRC_21_dataset = pyg.datasets.TUDataset(dataset_directory, "MSRC_21")
MSRC_21C_dataset = pyg.datasets.TUDataset(dataset_directory, "MSRC_21C")
MUTAG_dataset = pyg.datasets.TUDataset(dataset_directory, "MUTAG")
Mutagenicity_dataset = pyg.datasets.TUDataset(dataset_directory, "Mutagenicity")
NCI1_dataset = pyg.datasets.TUDataset(dataset_directory, "NCI1")
NCI109_dataset = pyg.datasets.TUDataset(dataset_directory, "NCI109")
PROTEINS_dataset = pyg.datasets.TUDataset(dataset_directory, "PROTEINS")
PTC_FM_dataset = pyg.datasets.TUDataset(dataset_directory, "PTC_FM")
PTC_FR_dataset = pyg.datasets.TUDataset(dataset_directory, "PTC_FR")
PTC_MM_dataset = pyg.datasets.TUDataset(dataset_directory, "PTC_MM")
PTC_MR_dataset = pyg.datasets.TUDataset(dataset_directory, "PTC_MR")
REDDIT_BINARY_dataset = pyg.datasets.TUDataset(dataset_directory, "REDDIT-BINARY")
SYNTHETICnew_dataset = pyg.datasets.TUDataset(dataset_directory, "SYNTHETICnew")
Synthie_dataset = pyg.datasets.TUDataset(dataset_directory, "Synthie")
Tox21_AR_dataset = pyg.datasets.TUDataset(dataset_directory, "Tox21_AR_training")
Tox21_AhR_dataset = pyg.datasets.TUDataset(dataset_directory, "Tox21_AhR_training")

PyObject Tox21_AhR_training(8169)

In [3]:
function eval_dataset(dataset, γ)
    completeness_ratio(x) = sum(x) / length(x)
    completeness_vec = zeros(length(dataset))
    for i in 0:length(dataset)-1
        edge_index = dataset[i].edge_index.numpy() .+ 1 
        if "x" in dataset[i].keys() x₀ = label_encoder(dataset[i].x.numpy())
        else x₀ = ones(dataset[i].num_nodes)
        end
        iters, coloring_wl = WeisfeilerLehman(x₀, edge_index)
        coloring_gnn = MPGNN(x₀, edge_index, γ, iters+1)
        completeness_vec[i+1] = sum(coloring_wl) <= sum(coloring_gnn)
    end
    return completeness_ratio(completeness_vec)
end

eval_dataset (generic function with 1 method)

In [35]:
setprecision(BigFloat, 50)
γ = rand(BigFloat)

println("success ratio AIDS: $(eval_dataset(AIDS_dataset, γ))%")
println("success ratio BZR: $(eval_dataset(BZR_dataset, γ))%")
println("success ratio COX2: $(eval_dataset(COX2_dataset, γ))%")
println("success ratio DHFR: $(eval_dataset(DHFR_dataset, γ))%")
println("success ratio DD: $(eval_dataset(DD_dataset, γ))%")
println("success ratio ENZYMES: $(eval_dataset(ENZYMES_dataset, γ))%")
println("success ratio FRANKENSTEIN: $(eval_dataset(FRANKENSTEIN_dataset, γ))%")
println("success ratio IMDB_BINARY: $(eval_dataset(IMDB_BINARY_dataset, γ))%")
println("success ratio IMDB_MULTI: $(eval_dataset(IMDB_MULTI_dataset, γ))%")
println("success ratio MSRC_9: $(eval_dataset(MSRC_9_dataset, γ))%")
println("success ratio MSRC_21: $(eval_dataset(MSRC_21_dataset, γ))%")
println("success ratio MSRC_21C: $(eval_dataset(MSRC_21C_dataset, γ))%")
println("success ratio MUTAG: $(eval_dataset(MUTAG_dataset, γ))%")
println("success ratio Mutagenicity: $(eval_dataset(Mutagenicity_dataset, γ))%")
println("success ratio NCI1: $(eval_dataset(NCI1_dataset, γ))%")
println("success ratio NCI109: $(eval_dataset(NCI109_dataset, γ))%")
println("success ratio PROTEINS: $(eval_dataset(PROTEINS_dataset, γ))%")
println("success ratio PTC_FM: $(eval_dataset(PTC_FM_dataset, γ))%")
println("success ratio PTC_FR: $(eval_dataset(PTC_FR_dataset, γ))%")
println("success ratio PTC_MM: $(eval_dataset(PTC_MM_dataset, γ))%")
println("success ratio PTC_MR: $(eval_dataset(PTC_MR_dataset, γ))%")
println("success ratio REDDIT_BINARY: $(eval_dataset(REDDIT_BINARY_dataset, γ))%")
println("success ratio SYNTHETICnew: $(eval_dataset(SYNTHETICnew_dataset, γ))%")
println("success ratio Synthie: $(eval_dataset(Synthie_dataset, γ))%")
println("success ratio Tox21_AR: $(eval_dataset(Tox21_AR_dataset, γ))%")
println("success ratio Tox21_AhR: $(eval_dataset(Tox21_AhR_dataset, γ))%")

success ratio IMDB_BINARY: 99.9%
success ratio IMDB_MULTI: 99.93333333333334%
success ratio MSRC_9: 100.0%
success ratio MSRC_21: 100.0%
success ratio MSRC_21C: 100.0%
success ratio MUTAG: 100.0%
success ratio Mutagenicity: 99.8616555222504%
success ratio NCI1: 99.7080291970803%
success ratio NCI109: 99.78192391567725%
success ratio PROTEINS: 99.73045822102426%
success ratio PTC_FM: 100.0%
success ratio PTC_FR: 100.0%
success ratio PTC_MM: 100.0%
success ratio PTC_MR: 100.0%
success ratio REDDIT_BINARY: 90.65%
success ratio SYNTHETICnew: 100.0%
success ratio Synthie: 100.0%
success ratio Tox21_AR: 99.88250373851741%
success ratio Tox21_AhR: 99.88982739625413%


In [4]:
setprecision(BigFloat, 50)

datasets_name = ["AIDS", "BZR", "COX2", "DHFR", "DD", "ENZYMES", "FRANKENSTEIN", "IMDB_BINARY", "IMDB_MULTI", "MSRC_9", "MSRC_21", "MSRC_21C", "MUTAG", "Mutagenicity", "NCI1", "NCI109", "PROTEINS", "PTC_FM", "PTC_FR", "PTC_MM", "PTC_MR", "REDDIT_BINARY", "SYNTHETICnew", "Synthie", "Tox21_AR", "Tox21_AhR"]
datasets = [AIDS_dataset, BZR_dataset, COX2_dataset, DHFR_dataset, DD_dataset, ENZYMES_dataset, FRANKENSTEIN_dataset, IMDB_BINARY_dataset, IMDB_MULTI_dataset, MSRC_9_dataset, MSRC_21_dataset, MSRC_21C_dataset, MUTAG_dataset, Mutagenicity_dataset, NCI1_dataset, NCI109_dataset, PROTEINS_dataset, PTC_FM_dataset, PTC_FR_dataset, PTC_MM_dataset, PTC_MR_dataset, REDDIT_BINARY_dataset, SYNTHETICnew_dataset, Synthie_dataset, Tox21_AR_dataset, Tox21_AhR_dataset]
γs = sort([rand(BigFloat) for _ in 1:10])
data = zeros(length(datasets), length(γs))

for i in 1:length(datasets)
    for j in 1:length(γs)
        data[i, j] = eval_dataset(datasets[i], γs[j])
    end
end

In [5]:
df = DataFrame()
df[!, "Datasets"] = datasets_name
for i in 1:length(γs)
    df[!, string(γs[i])] = data[1:end, i]
end
df

Row,Datasets,0.047632445456408767,0.050724755360278095,0.19918206223757995,0.29442653835248134,0.41964098075459066,0.45084110661256549,0.75045566955509369,0.86647638573282482,0.87574121960825835,0.97318473920207271
,String,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64
1,AIDS,0.981,0.981,0.9975,0.998,0.999,0.999,0.999,0.9985,0.9985,0.998
2,BZR,0.997531,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
3,COX2,0.925054,0.946467,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
4,DHFR,0.981481,0.982804,0.996032,0.998677,1.0,0.998677,0.998677,0.996032,0.996032,0.993386
5,DD,1.0,1.0,1.0,1.0,0.995756,0.991511,0.668081,0.380306,0.353141,0.188455
6,ENZYMES,0.976667,0.98,0.991667,0.993333,0.996667,0.995,0.991667,0.988333,0.988333,0.983333
7,FRANKENSTEIN,0.980401,0.982707,0.998617,0.998847,0.999308,0.999308,0.999078,0.999078,0.999078,0.999078
8,IMDB_BINARY,1.0,1.0,1.0,1.0,0.999,0.999,0.997,0.994,0.994,0.991
9,IMDB_MULTI,1.0,1.0,1.0,1.0,1.0,1.0,0.994667,0.992667,0.992667,0.988667


In [206]:
setprecision(BigFloat, 5)
heatmap(γs, datasets_name, data,  
    clim=(0, 1),
    c=cgrad(:RdYlGn, 10, categorical = true),
    xticks= [1 for j in 1:10],
    yticks = :all,
    yflip=true,
    axis=true,
    framestyle = :box,
    dpi=1000)

title!("WL-completeness ratio")
xlabel!("γs")
ylabel!("Datasets")
annotate!([(j/10.15-0.01, i-0.45, text(round(data[i,j], digits=2), 6, "Computer Modern", :black)) for i in 1:26 for j in 1:10])
annotate!([(j/10.15-0.01, i-0.25, text(round(γs[j], digits=3), 8, "Arial", :black)) for i in 27:27 for j in 1:10])
Plots.savefig("results\\5.completeness_rate.png")
println()

In [97]:
using DelimitedFiles 
writedlm("results\\5.completeness_rate.csv", Iterators.flatten(([names(df)], eachrow(df))), ',')